In [0]:
# !cd /content
# !pwd
# !ls /content

In [0]:
# !pip install jieba
# !pip install gensim

In [1]:
# import pdb

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

data_root = '/gdrive/My Drive/data_tf'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
# Above is for notebook only
# Below is to be moved to py files
! ls '/gdrive/My Drive/python-code/文本摘要-01'

data  utils  work.ipynb  work-tf-unigru-attn.ipynb


# 上面是测试用
# ============================================================
# 下面要转移到文件里面

In [3]:
# import modules
import os
import sys
import pathlib
import pandas as pd
import numpy as np
import pdb
import time

project_root = pathlib.Path(r'/gdrive/My Drive/python-code/文本摘要-01')
sys.path.append(str(project_root))

from utils.config import *
from utils.multi_proc_utils import *
from utils.data_loader import *
from utils.file_utils import *
from utils.data_preprocessor import *

ImportError: ignored

# Create data preprocessor

In [4]:
wv_train_epochs = 1
wv_update_epochs = 1

working_ds = AutoCarDataSet()
working_ds.prepare_data(force_build=False)
working_ds.get_wv_model(force_build=False)

NameError: ignored

# 以下是tensorflow实现带attention 的seq2seq=============

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
# define constants for codec
BUFFER_SIZE = working_ds.train_ids_x.shape[0]
BATCH_SIZE = 16
steps_per_epoch = working_ds.train_ids_x.shape[0]//BATCH_SIZE
embedding_dim = embedding_dim
codec_units = 256
vocab_inp_size = len(working_ds.vocab)
vocab_tar_size = len(working_ds.vocab)


TensorFlow 2.x selected.


In [0]:
class UnidirGruEncoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, word_matrix, enc_units, batch_sz):
    super(UnidirGruEncoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [word_matrix])
    self.gru = tf.keras.layers.GRU(self.enc_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform')
    
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

  def call(self, x, hidden):
    # debug_print("input x.shape 1: ", x.shape)
    # debug_print("input hidden: ", hidden.shape)
    
    x = self.embedding(x)
    # debug_print("input x.shape 2: ", x.shape)
    
    output, state = self.gru(x, initial_state = hidden)
    # debug_print("output.shape: ", output.shape)
    # debug_print("state.shape: ", state.shape)
    
    return output, state

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # debug_print("query.shape; ", query.shape)
    # debug_print("values.shape; ", values.shape)

    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)
    # debug_print("hidden_with_time_axis.shape; ", hidden_with_time_axis.shape)

    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    w1 = self.W1(values)
    # debug_print("w1.shape; ", w1.shape)

    w2 = self.W2(hidden_with_time_axis)
    # debug_print("w2.shape; ", w2.shape)

    sum_w12 = w1 + w2
    # debug_print("sum_w12.shape; ", sum_w12.shape)

    aw12 = tf.nn.tanh(sum_w12)
    # debug_print("aw12.shape; ", aw12.shape)

    score = self.V(aw12)
    # debug_print("score.shape; ", score.shape)

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)
    # debug_print("attention_weights.shape; ", attention_weights.shape)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    # debug_print("context_vector.shape; ", context_vector.shape)

    context_vector = tf.reduce_sum(context_vector, axis=1)
    # debug_print("context_vector.shape; ", context_vector.shape)

    return context_vector, attention_weights

In [0]:
class UniGruDecoderWithAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, word_matrix, dec_units, batch_sz):
    super(UniGruDecoderWithAttention, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [word_matrix])
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    context_vector = tf.expand_dims(context_vector, 1)
    # debug_print("context_vector expanded shape; ", context_vector.shape)

    x = tf.concat([context_vector, x], axis=-1)
    # debug_print("x concated context_vector shape; ", x.shape)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)
    # debug_print("gru output.shape; ", output.shape)
    # debug_print("gru state.shape; ", state.shape)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))
    # debug_print("fc input shape; ", output.shape)

    # output shape == (batch_size, vocab)
    x = self.fc(output)
    # debug_print("fc output shape; ", x.shape)

    return x, state, attention_weights

In [0]:
optimizer = tf.keras.optimizers.Adam(1e-2)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))

    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [0]:
def train_one_batch(encoder, decoder, inp, targ, enc_hidden):
  loss = 0

  # debug_print("train_step inp.shape: ", inp.shape)
  # debug_print("train_step targ.shape: ", targ.shape)
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([working_ds.vocab['<START>']] * BATCH_SIZE, 1)
    # debug_print("train_step dec_input.shape: ", dec_input.shape)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  # debug_print("train_step predictions.shape: ", predictions.shape)
  # debug_print("train_step loss.shape: ", loss.shape)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables
  # debug_print("train_step variables.shape: ", variables.shape)

  gradients = tape.gradient(loss, variables)
  # debug_print("train_step gradients.shape: ", gradients.shape)

  zipped_variables = zip(gradients, variables)
  # debug_print("zipped_variables.shape: ", zipped_variables.shape)
  optimizer.apply_gradients(zipped_variables)

  return batch_loss


In [0]:
#@tf.function
def train_one_epoch(encoder, decoder, dataset):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0.0

    starttime = time.time()
    print('Epoch start time {}'.format(starttime))
    loss_history = []
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_one_batch(encoder, decoder, inp, targ, enc_hidden)
        # pdb.set_trace()
        total_loss += batch_loss

        loss_history.append(batch_loss)

        # if batch == 5:
        #   break

        if (batch+1) % 10 == 0:
            # print('Batch {} time {:.2f}'.format(batch, time.time() - starttime))
            print('Batch {} Loss {:.4f} time {:.2f}'.format(batch+1, batch_loss.numpy(), time.time() - starttime)) #Not valid if @tf.function
    print('Loss at epoch end {:.4f}'.format(total_loss / steps_per_epoch)) #Not valid if @tf.function

    return total_loss / steps_per_epoch

In [0]:
def def_checkpoint(encoder, decoder, optimizer):
    # checkpoint_dir = checkpoint_dir_tf
    # checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_unigru_attn")
    checkpoint = tf.train.Checkpoint(encoder=encoder, decoder=decoder, optimizer=optimizer)
    return checkpoint

def create_codec():
    encoder = UnidirGruEncoder(vocab_inp_size, embedding_dim, working_ds.embedding_matrix, codec_units, BATCH_SIZE)
    decoder = UniGruDecoderWithAttention(vocab_tar_size, embedding_dim, working_ds.embedding_matrix, codec_units, BATCH_SIZE)
    return encoder, decoder

def get_dataset():
    dataset = tf.data.Dataset.from_tensor_slices((working_ds.train_ids_x, working_ds.train_ids_y)).shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

def train_all(encoder, decoder, dataset, epochs, save_chkpnts):

    if save_chkpnts:
        checkpoint = def_checkpoint(encoder, decoder, optimizer)
    else:
        checkpoint = None

    loss_history = []
    for e in range(epochs):
        print('Epoch {} ============================>'.format(e))
        epoch_loss = train_one_epoch(encoder, decoder, dataset)
        loss_history.append(epoch_loss)
        
        if checkpoint  and (e + 1) % 1 == 0:
            # saving (checkpoint) the model every 2 epochs
            checkpoint.save(file_prefix = checkpoint_prefix)

    return loss_history

In [0]:
def document_as_tensor(doc):
    doc = sentence_proc(doc)
    doc = pad_proc(doc, working_ds.X_max_len, working_ds.vocab)
    doc = transform_data(doc, working_ds.vocab)
    doc = tf.convert_to_tensor(doc)
    return doc

def evaluate(encoder, decoder, doc):
    attention_plot = np.zeros((working_ds.X_max_len, working_ds.train_y_max_len))

    inputs = document_as_tensor(doc)

    result = ''

    hidden = [tf.zeros((1, codec_units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([working_ds.vocab['<START>']], 0)

    for t in range(working_ds.train_y_max_len):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += working_ds.reverse_vocab[predicted_id] + ' '

        if working_ds.reverse_vocab[predicted_id] == '<STOP>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, inputs, attention_plot

In [0]:
def train_for_restore(encoder, decoder, dataset):
  enc_hidden = encoder.initialize_hidden_state()

  inp, targ = next(iter(dataset))
  batch_loss = train_one_batch(encoder, decoder, inp, targ, enc_hidden)

  loss = 0
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([working_ds.vocab['<START>']] * BATCH_SIZE, 1)
    # debug_print("train_step dec_input.shape: ", dec_input.shape)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  zipped_variables = zip(gradients, variables)
  optimizer.apply_gradients(zipped_variables)

  return batch_loss


# 这里是上下分隔符，下面都是测试代码=======================

In [0]:
encoder, decoder = create_codec()

dataset = get_dataset()
loss_history = train_all(encoder, decoder, dataset, 1, True)
encoder.summary()
decoder.summary()


Epoch 0 ============================>
Epoch start time 1576832240.597346
Batch 10 Loss 3.8703 time 15.32
Batch 20 Loss 3.0178 time 24.06
Batch 30 Loss 2.3273 time 32.69
Batch 40 Loss 2.1977 time 41.20
Batch 50 Loss 2.5271 time 49.98
Batch 60 Loss 2.6982 time 58.90
Batch 70 Loss 2.3343 time 67.66
Batch 80 Loss 2.6145 time 76.07
Batch 90 Loss 2.4875 time 84.76
Batch 100 Loss 2.6055 time 93.66
Batch 110 Loss 2.2349 time 101.85
Batch 120 Loss 2.4149 time 110.69
Batch 130 Loss 2.5225 time 119.37
Batch 140 Loss 1.7834 time 128.22
Batch 150 Loss 2.3185 time 136.49
Batch 160 Loss 2.4661 time 145.19
Batch 170 Loss 2.6094 time 153.91
Batch 180 Loss 3.0307 time 162.53
Batch 190 Loss 2.6528 time 171.19
Batch 200 Loss 2.2771 time 179.96
Batch 210 Loss 2.8102 time 188.67
Batch 220 Loss 3.5790 time 196.89
Batch 230 Loss 2.2124 time 205.89
Batch 240 Loss 2.4412 time 214.65
Batch 250 Loss 2.1005 time 223.03
Batch 260 Loss 2.2075 time 231.71
Batch 270 Loss 2.3308 time 240.60
Batch 280 Loss 2.0358 time 2

In [0]:
encoder, decoder = create_codec()
train_for_restore(encoder, decoder, get_dataset())

<tf.Tensor: shape=(), dtype=float32, numpy=8.390885>

In [0]:
encoder.trainable_variables

In [0]:
decoder.trainable_variables

In [0]:
print(checkpoint_prefix)
print(checkpoint_dir)

In [0]:
checkpoint = def_checkpoint(encoder, decoder, optimizer)

load_status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
load_status.assert_consumed()

In [0]:
decoder.embedding.embeddings

In [0]:
import os
from tensorflow.python import pywrap_tensorflow
# checkpoint_path = os.path.join(model_dir, "model.ckpt")
checkpoint_path = checkpoint_prefix
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print("\r\ntensor_name: ", key, end=' ')
    print(reader.get_tensor(key))


In [0]:
load_status.assert_existing_objects_matched()

In [0]:
doc = "更好机油散热器，问题依然没有解决技师说仪表亮红色机油灯？|车主说|车主说滴滴几声恢复正常。请参考原先提问！|技师说到哪去找。|车主说图片|车主说看到？|技师说领驭？|车主说|车主说感觉行驶一段时间后报警。走走停停报警次数增多。|技师说稍|技师说图片|技师说图片|技师说测量一下标准机油压力，压力低需要进行相应检查维修。|车主说凉车机油压力4|车主说热车才1|技师说热车怠速最低1.3|车主说不了1.3|技师说发动机水温？|车主说水温正常|车主说没有测量温度|技师说水温80度机油压力不到1.3，说明卸压，水温90以上，不到1.3说明散热不好。|车主说好|车主说再试试|车主说两种情况处理？|技师说冷车机油粘度大，压力肯定高|车主说两种情况解决？|技师说水温80度，机油压力灯亮，说明机械方面故障，考虑机油泵，限压阀，机油喷嘴，热车机油压力灯亮，重点考虑机油粘度机油散热器，水箱是否堵塞，机油里加注抗磨剂。"
ts = document_as_tensor(doc)
evaluate(encoder, decoder, doc)

In [0]:
ts

In [0]:
decoder = UniGruDecoderWithAttention(vocab_tar_size, embedding_dim, working_ds.embedding_matrix, codec_units, BATCH_SIZE)

sample_decoder_output, state, attention_weights = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)
decoder.summary()
print ('UniGruDecoderWithAttention output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
print ('UniGruDecoderWithAttention state shape: (batch_size, vocab size) {}'.format(state.shape))
print ('UniGruDecoderWithAttention attention_weights shape: (batch_size, vocab size) {}'.format(attention_weights.shape))

In [0]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)
attention_layer.summary()

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [0]:
21195*300

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((working_ds.train_ids_x, working_ds.train_ids_y)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape, example_target_batch.shape)
print(example_input_batch)

encoder = UnidirGruEncoder(vocab_inp_size, embedding_dim, working_ds.embedding_matrix, codec_units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

encoder.summary()
print ('ThisEncoder input shape: (batch size, sequence length) {}'.format(example_input_batch.shape))
print ('ThisEncoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('ThisEncoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))
print(type(encoder.embedding.embeddings))

In [0]:
# !pip show tensorflow-gpu
# tf.__version__
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None )

In [0]:
# -*- coding:utf-8 -*-
# Created by Kevin Luo at 2019-12-17
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


def seq2seq(input_length, output_sequence_length, embedding_matrix, vocab_size):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], trainable=False,
                        input_length=input_length))
    model.add(Bidirectional(GRU(300, return_sequences=False)))
    model.add(Dense(300, activation="relu"))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(300, return_sequences=True)))
    model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(1e-3))
    model.summary()
    return model


In [0]:
print(working_ds.train_df['X'].iloc[4999])

working_ds.wv_model.most_similar(positive=['语音'], negative=[])

print(working_ds.train_csv)
print(working_ds.test_csv)
        
print(working_ds.train_df.shape)
print(working_ds.test_df.shape)
print(working_ds.merged_df.shape)
        
print(working_ds.train_y_max_len)
        
print(working_ds.X_max_len)
		
print(working_ds.train_X.shape)
print(working_ds.train_Y.shape)
print(working_ds.test_X.shape)

print(working_ds.train_ids_x.shape)
print(working_ds.train_ids_y.shape)
print(working_ds.test_ids_x.shape)
		
print(working_ds.wv_model)
		
print(len(working_ds.vocab))
print(len(working_ds.reverse_vocab))
		
print(working_ds.embedding_matrix.shape)

print(working_ds.train_ids_x.shape)
print(working_ds.train_ids_y.shape)
print(working_ds.test_ids_x.shape)


In [0]:
basic_s2s_model = seq2seq(working_ds.X_max_len, working_ds.train_y_max_len, working_ds.embedding_matrix, len(working_ds.vocab))

basic_s2s_model.fit(working_ds.train_X, working_ds.train_Y, batch_size=32, epochs=1, validation_split=0.2)
basic_s2s_model.save(os.path.join(project_root, r'data\seq2seq_basic_s2s.model'))

In [0]:
working_ds.wv_model.most_similar(positive=[''], negative=[])

In [0]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))